# Segmentação de imagens médicas - Notebook responsavel pela criação e treinamento da rede neural

# Importação das bibliotecas

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow
from tensorflow.keras.optimizers.legacy import Adam
import random
from glob import glob
from imageio import mimread
from google.colab.patches import cv2_imshow
import tensorflow_datasets as tfds
tensorflow.__version__

'2.11.0'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Carregamento da base de dados

In [ ]:
def carregar_dataset(caminho):
  X_train = sorted(glob(os.path.join(caminho, "train", "principal", "*.png")))
  y_train = sorted(glob(os.path.join(caminho, "train", "mask", "*.jpg")))

  X_test = sorted(glob(os.path.join(caminho, "test", "principal", "*.png")))
  y_test = sorted(glob(os.path.join(caminho, "test", "mask", "*.jpg")))

  return (X_train, y_train), (X_test, y_test)

In [ ]:
caminho = '/content/gdrive/MyDrive/Segmentacao/medical-images'

In [ ]:
(X_train, y_train), (X_test, y_test) = carregar_dataset(caminho)

In [ ]:
len(X_train), len(y_train)

(80, 80)

In [ ]:
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

/content/gdrive/MyDrive/Segmentacao/medical-images/train/principal/ID_0001_AGE_0069_CONTRAST_1_CT.png
/content/gdrive/MyDrive/Segmentacao/medical-images/train/mask/MASK_ID_0001_AGE_0069_CONTRAST_1_CT.jpg
/content/gdrive/MyDrive/Segmentacao/medical-images/test/principal/ID_0000_AGE_0060_CONTRAST_1_CT.png
/content/gdrive/MyDrive/Segmentacao/medical-images/test/mask/MASK_ID_0000_AGE_0060_CONTRAST_1_CT.jpg


In [ ]:
len(X_test), len(y_test)

(20, 20)

In [ ]:
img_altura = 512
img_largura = 512

In [ ]:
epochs = 50
batch_size = 2
lr = 1e-4

# Construção da rede neural

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.metrics import Recall, Precision

def bloco_conv(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x) 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x) 
    x = Activation("relu")(x)

    return x

def bloco_encoder(input, num_filters):
    x = bloco_conv(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p  


def bloco_decoder(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = bloco_conv(x, num_filters)
    return x 


def modelo_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = bloco_encoder(inputs, 64)
    s2, p2 = bloco_encoder(p1, 64*2) #128
    s3, p3 = bloco_encoder(p2, 64*4) #256
    s4, p4 = bloco_encoder(p3, 64*8) #512

    b1 = bloco_conv(p4, 64*16)  #1024

    d1 = bloco_decoder(b1, s4, 64*8) #512
    d2 = bloco_decoder(d1, s3, 64*4) #256
    d3 = bloco_decoder(d2, s2, 64*2) #128
    d4 = bloco_decoder(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)  

    model = Model(inputs, outputs, name="UNet")
    return model

In [ ]:
from keras import backend as K

""" IoU """
def iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

""" Dice Coefficient """
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

""" Dice Coefficient Loss """
def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [ ]:
model = modelo_unet((img_altura, img_largura, 3))
model.compile(loss=dice_coef_loss, optimizer=Adam(lr), metrics = [dice_coef, iou, 'accuracy'])
model.summary()

Model: "UNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

In [ ]:
def ler_img_dataset(caminho):
  caminho = caminho.decode()
  img = cv2.imread(caminho)  
  img = img / 255.0
  img = img.astype(np.float32)
  return img

In [ ]:
def ler_mask_dataset(caminho):
  caminho = caminho.decode()
  img = cv2.imread(caminho, cv2.IMREAD_GRAYSCALE)
  img = img / 255.0
  img = img.astype(np.float32)
  img = np.expand_dims(img, axis = -1) # (512, 512) -> (512, 512, 1)
  return img

In [ ]:
def tf_parse(x, y):
  def _parse(x, y):
    x = ler_img_dataset(x)
    y = ler_mask_dataset(y)
    return x, y

  x, y = tensorflow.numpy_function(_parse, [x, y], [tensorflow.float32, tensorflow.float32])
  x.set_shape([img_altura, img_largura, 3])
  y.set_shape([img_altura, img_largura, 1])
  return x, y

In [ ]:
def tf_dataset(X, y, batch_size=2):
  dataset = tensorflow.data.Dataset.from_tensor_slices((X, y))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(4)
  return dataset

In [ ]:
dataset_train = tf_dataset(X_train, y_train, batch_size=batch_size)
dataset_val = tf_dataset(X_test, y_test, batch_size=batch_size)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
callbacks = [ModelCheckpoint('/content/gdrive/MyDrive/Segmentacao/modelo' + '/modelo_drive_{epoch:02d}.h5', verbose=1, save_best_only=True),
             EarlyStopping(monitor='val_loss', patience=5)]

# Treinamento da rede neural para predição das imagens

In [ ]:
history = model.fit(dataset_train, epochs=epochs, validation_data=dataset_val, callbacks=callbacks)

Epoch 1/50
40/40 [==============================] - ETA: 0s - loss: 0.3173 - dice_coef: 0.6827 - iou: 0.5219 - accuracy: 0.4290
Epoch 1: val_loss improved from inf to 0.37655, saving model to /content/gdrive/MyDrive/Segmentacao/modelo/modelo_drive_01.h5
40/40 [==============================] - 87s 1s/step - loss: 0.3173 - dice_coef: 0.6827 - iou: 0.5219 - accuracy: 0.4290 - val_loss: 0.3766 - val_dice_coef: 0.6234 - val_iou: 0.4533 - val_accuracy: 0.4536
Epoch 2/50
40/40 [==============================] - ETA: 0s - loss: 0.2371 - dice_coef: 0.7629 - iou: 0.6186 - accuracy: 0.4540
Epoch 2: val_loss improved from 0.37655 to 0.32472, saving model to /content/gdrive/MyDrive/Segmentacao/modelo/modelo_drive_02.h5
40/40 [==============================] - 27s 685ms/step - loss: 0.2371 - dice_coef: 0.7629 - iou: 0.6186 - accuracy: 0.4540 - val_loss: 0.3247 - val_dice_coef: 0.6753 - val_iou: 0.5104 - val_accuracy: 0.4535
Epoch 3/50
40/40 [==============================] - ETA: 0s - loss: 0.2108 